In [ ]:
!pip install trl -q
!pip install -U bitsandbytes -q

In [ ]:
from google.colab import drive
import os
# Monter Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os

# Chemin vers ton fichier
file_path = '/content/drive/MyDrive/yelptrain_complete.csv'

# Charger le CSV
df = pd.read_csv(file_path)

# Vérification rapide
print(f"Shape du dataset : {df.shape}")
df.head()

Shape du dataset : (29514, 7)


,review_id,user_id,business_id,stars,text,summary,summary_completed
0,PlhUBK-5oNu-VILpHAHX2g,-FxsSuwDbIII7yo5BjHpiA,3hrEWKJxJUwEBmzw8oUYHw,4,im a europhile a europhile is a person who is ...,Brugges is the place to go with some friends f...,Brugges is the place to go with some friends f...
1,VL29c_fFS8vxtEbF8EI0AQ,-FxsSuwDbIII7yo5BjHpiA,_8MkUoObdLpakXbCobbKxA,3,"wikipedia definition of ok as an adjective, ok...","The food, the drinks, the service, the feel ev...","The food, the drinks, the service, the feel ev..."
2,Y-OoUSRjMowD1y0_HBH__w,-FxsSuwDbIII7yo5BjHpiA,b13EoceGelGyrIPRxxiERQ,5,we first tried wild eggs in louisville and lov...,Luxury eatery is located by mass ave. Offers a...,Luxury eatery is located by mass ave. Offers a...
3,iUtp75SGJ8Tz3nzdDDwrvA,-FxsSuwDbIII7yo5BjHpiA,S_jfDqc2TUhkraTpeZgqjg,2,i feel like i have missed something here. so s...,i feel like i have missed something here. the ...,i feel like i have missed something here. the ...
4,w213SIcQi7YrUyiNjYC5qA,-FxsSuwDbIII7yo5BjHpiA,IcqvNEGpi94KcvpS_wUBRQ,5,i had the good fortune to get a preview of the...,i had the good fortune to get a preview of the...,i had the good fortune to get a preview of the...


In [ ]:
# import pandas as pd
# import numpy as np

# # Moyenne globale
# global_mean = df['stars'].mean()

# # Biais utilisateur
# mean_user = df.groupby('user_id')['stars'].mean()
# bias_user = mean_user - global_mean

# # Biais business
# mean_business = df.groupby('business_id')['stars'].mean()
# bias_business = mean_business - global_mean

# print("Moyenne globale :", global_mean)
# print("Biais utilisateur :\n", bias_user.head())
# print("Biais business :\n", bias_business.head())

In [ ]:
import pandas as pd
import os

# Chemin vers ton fichier
file_path = '/content/drive/MyDrive/colab_pro/yelptest_complete.csv'

# Charger le CSV
df = pd.read_csv(file_path)

# Vérification rapide
print(f"Shape du dataset : {df.shape}")
df.head()

Shape du dataset : (7680, 7)


,review_id,user_id,business_id,stars,text,summary,summary_completed
0,KsJ-MT-dC_E4_Z-kG3w7Pw,-FxsSuwDbIII7yo5BjHpiA,6x6rR-SErwOo3xF2AzXVHA,5,our museum is such a local treasure. i attende...,our museum is such a local treasure. i attende...,our museum is such a local treasure. i attende...
1,TXiT7eHgsZh2al_Yn-_gsg,-FxsSuwDbIII7yo5BjHpiA,3y61A28PQDZ4weeeTRfIlA,4,"update upon a return visit to burger study, i ...",bacon was cut in thick little salty cubes that...,bacon was cut in thick little salty cubes that...
2,AFpiBRLNCTEHG03JcBzv7A,-FxsSuwDbIII7yo5BjHpiA,bsumxuX4XOeJGTmc_ttm7g,3,soup is good food sometimes. ive been to soupr...,soup is good food sometimes. ive been to soupr...,soup is good food sometimes. ive been to soupr...
3,3s8rXBszWxHsjvb1IaxJgQ,-FxsSuwDbIII7yo5BjHpiA,00rY5F9ltW-IWf2Ev96kOg,5,"its tucked away on th street along the canal, ...","its tucked away on th street along the canal, ...","its tucked away on th street along the canal, ..."
4,PJ4RXbNxYqjBi4GA6kTIAQ,-FxsSuwDbIII7yo5BjHpiA,gsy3nze2K5cSH9H6rOkTgQ,5,longs bakery is one of the indianapolis greats...,longs bakery is one of the indianapolis greats...,longs bakery is one of the indianapolis greats...


In [ ]:
from huggingface_hub import login
login("---")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Configuration pour chargement en 4-bit avec bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Important pour les modèles causaux

# Modèle
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt_template_yelp = """You are a rating prediction expert. Based on the review text and user/business information, predict the rating (stars) the user will give.
Return only the rating as a single integer from 1 to 5, with no additional text.

Review Info: review_id={review_id}, user_id={user_id}, business_id={business_id}
Review Text: "{summary_completed}"

### Response:
"""

In [ ]:
def format_prompt_yelp(example):
    prompt = prompt_template_yelp.format(
        review_id=example['review_id'],
        user_id=example['user_id'],
        business_id=example['business_id'],
        summary_completed=example['summary_completed']
    ).rstrip()
    label = str(int(example['stars']))  # La completion contiendra uniquement le rating
    return {"prompt": prompt, "completion": label}

dataset_yelp = [format_prompt_yelp(row) for _, row in df.iterrows()]

In [ ]:
import pandas as pd

# Charger ton dataset final déjà sauvegardé
test_df = pd.read_csv("/content/drive/MyDrive/colab_pro/yelptest_complete.csv")
train_df = pd.read_csv("/content/drive/MyDrive/yelptrain_complete.csv")

# Appliquer format_prompt_yelp
test_dataset = test_df.apply(format_prompt_yelp, axis=1)
train_dataset = train_df.apply(format_prompt_yelp, axis=1)

# Convertir en DataFrame formaté
test_formatted = pd.DataFrame(list(test_dataset))
train_formatted = pd.DataFrame(list(train_dataset))

# Vérifier un aperçu
print(test_formatted.head())
print(train_formatted.head())

                                              prompt completion
0  You are a rating prediction expert. Based on t...          5
1  You are a rating prediction expert. Based on t...          4
2  You are a rating prediction expert. Based on t...          3
3  You are a rating prediction expert. Based on t...          5
4  You are a rating prediction expert. Based on t...          5
                                              prompt completion
0  You are a rating prediction expert. Based on t...          4
1  You are a rating prediction expert. Based on t...          3
2  You are a rating prediction expert. Based on t...          5
3  You are a rating prediction expert. Based on t...          2
4  You are a rating prediction expert. Based on t...          5


In [ ]:
from datasets import Dataset, DatasetDict

# Créer le Dataset test
train_ds = Dataset.from_pandas(train_formatted)
test_ds = Dataset.from_pandas(test_formatted)

# Créer le DatasetDict
dataset = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 29514
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 7680
    })
})


In [ ]:
import os
os.listdir("/content/drive/MyDrive/colab_pro/yelp_fine_tuned")

['checkpoint-231',
 'checkpoint-462',
 'README.md',
 'special_tokens_map.json',
 'tokenizer_config.json',
 'chat_template.jinja',
 'tokenizer.model',
 'tokenizer.json']

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/colab_pro/yelp_fine_tuned/checkpoint-462",
    device_map="auto",
    torch_dtype=torch.float16
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#  Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import GenerationConfig

#  Mettre le modèle en mode évaluation
model.eval()
# generation_config = GenerationConfig(
#     max_new_tokens=10,        # juste assez pour la note
#     do_sample=True,
#     top_k=5,                  # moins dispersé
#     top_p=0.8,
#     temperature=0.6,          # plus stable
#     repetition_penalty=1.2,
#     pad_token_id=tokenizer.pad_token_id,
#     eos_token_id=tokenizer.eos_token_id
# )
generation_config = GenerationConfig(
    max_new_tokens=10,        # juste assez pour la note
    do_sample=False,          # désactive l’échantillonnage → déterministe
    top_k=None,               # inutile si do_sample=False
    top_p=None,               # idem
    temperature=None,         # idem
    repetition_penalty=1.0,   # tu peux le laisser à 1.0 pour neutre
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)


In [ ]:
import re

def clean_prediction(pred_text, fallback_value=None):
    if not pred_text:
        return fallback_value

    # Nettoyer texte
    pred_text = pred_text.strip().replace("\n", " ")
    pred_text = re.sub(r"[^a-zA-Z0-9\s]", "", pred_text.lower())

    # Chercher chiffre 1-5
    match = re.search(r"(?:### )?Response[:\s]*(?:is\s*)?([1-5])\b", pred_text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Chercher nombres en lettres (FR/EN)
    word_to_digit = {
        "un":1, "deux":2, "trois":3, "quatre":4, "cinq":5,
        "one":1, "two":2, "three":3, "four":4, "five":5
    }
    for word, digit in word_to_digit.items():
        if re.search(rf"\b{word}\b", pred_text):
            return digit

    # fallback
    return fallback_value

In [ ]:
import json
from tqdm import tqdm
import re
import torch

def clean_prediction(pred_text, fallback_value=-1):
    if not pred_text:
        return fallback_value

    # Nettoyer texte
    pred_text = pred_text.strip().replace("\n", " ")
    pred_text = re.sub(r"[^a-zA-Z0-9\s]", "", pred_text.lower())

    # Chercher chiffre 1-5
    match = re.search(r"(?:### )?Response[:\s]*(?:is\s*)?([1-5])\b", pred_text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Chercher nombres en lettres
    word_to_digit = {
        "un":1, "deux":2, "trois":3, "quatre":4, "cinq":5,
        "one":1, "two":2, "three":3, "four":4, "five":5
    }
    for word, digit in word_to_digit.items():
        if word in pred_text:
            return digit

    return fallback_value

batch_size = 40
test_predictions = []
true_labels = []

max_display = 70
displayed = 0
max_retries = 3

for start_idx in tqdm(range(0, len(test_formatted), batch_size), desc="Inference batches"):
    batch = test_formatted.iloc[start_idx:start_idx+batch_size]
    prompts = batch['prompt'].tolist()

    # Tokenisation + envoi sur GPU
    inputs = {k: v.to(model.device) for k, v in tokenizer(prompts, return_tensors="pt", truncation=True, padding=True, max_length=1024).items()}

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            generation_config=generation_config
        )

    for i, output in enumerate(outputs):
        prediction = tokenizer.decode(output, skip_special_tokens=True).strip()
        pred_note = clean_prediction(prediction)

        retries = 0
        # Retry si valeur invalide
        while pred_note not in [1,2,3,4,5] and retries < max_retries:
            retries += 1
            with torch.no_grad():
                new_output = model.generate(
                    input_ids=inputs['input_ids'][i].unsqueeze(0),
                    attention_mask=inputs['attention_mask'][i].unsqueeze(0),
                    generation_config=generation_config
                )
            prediction = tokenizer.decode(new_output[0], skip_special_tokens=True).strip()
            pred_note = clean_prediction(prediction)

        if displayed < max_display:
            print(f"Prompt:\n{prompts[i]}")
            print(f"Prediction brute: {prediction}")
            print(f"Note extraite: {pred_note}\n")
            displayed += 1

        test_predictions.append(pred_note)
        true_labels.append(int(batch.iloc[i]['completion']))

Inference batches:   1%|          | 1/192 [00:04<13:17,  4.18s/it]

Prompt:
You are a rating prediction expert. Based on the review text and user/business information, predict the rating (stars) the user will give.
Return only the rating as a single integer from 1 to 5, with no additional text.

Review Info: review_id=KsJ-MT-dC_E4_Z-kG3w7Pw, user_id=-FxsSuwDbIII7yo5BjHpiA, business_id=6x6rR-SErwOo3xF2AzXVHA
Review Text: "our museum is such a local treasure. i attended an event here, and was able to see what a great facility it is to host events. the toby tobias theater has ample, comfortable seating with every audio visual tool available that one could need for a symposium. they have spacious rooms for seated dinners or even dancing. they are able to provide bar service and catering fit for any gala."

### Response:
Prediction brute: You are a rating prediction expert. Based on the review text and user/business information, predict the rating (stars) the user will give.
Return only the rating as a single integer from 1 to 5, with no additional text.

R

Inference batches:   1%|          | 2/192 [00:08<13:04,  4.13s/it]

Prompt:
You are a rating prediction expert. Based on the review text and user/business information, predict the rating (stars) the user will give.
Return only the rating as a single integer from 1 to 5, with no additional text.

Review Info: review_id=27QBr85oRHch59oapZ6atw, user_id=-KduD3wwydWDq2GrODh2tw, business_id=a3YGrsA5i1JSgs7g0tOefw
Review Text: "lovely, peaceful area in garfield park. not out of place in many worldclass cities. Not out of the way of the city centre."

### Response:
Prediction brute: You are a rating prediction expert. Based on the review text and user/business information, predict the rating (stars) the user will give.
Return only the rating as a single integer from 1 to 5, with no additional text.

Review Info: review_id=27QBr85oRHch59oapZ6atw, user_id=-KduD3wwydWDq2GrODh2tw, business_id=a3YGrsA5i1JSgs7g0tOefw
Review Text: "lovely, peaceful area in garfield park. not out of place in many worldclass cities. Not out of the way of the city centre."

### Response

Inference batches: 100%|██████████| 192/192 [12:10<00:00,  3.80s/it]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse_test = np.sqrt(mean_squared_error(true_labels, test_predictions))
mae_test = mean_absolute_error(true_labels, test_predictions)

print(f"RMSE test : {rmse_test:.4f}")
print(f"MAE test  : {mae_test:.4f}")

RMSE test : 0.6430
MAE test  : 0.3759


In [ ]:
# !pip install sentence-transformers

In [ ]:
# print(train_df.columns)
# print(test_df.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'text', 'summary',
       'summary_completed'],
      dtype='object')
Index(['review_id', 'user_id', 'business_id', 'stars', 'text', 'summary',
       'summary_completed', 'pred_note'],
      dtype='object')


In [ ]:
from sklearn.metrics import roc_auc_score

# Transformer les vraies notes en binaire (1 si >=4, 0 sinon)
binary_test_labels = [1 if label >= 4 else 0 for label in true_labels]

# Transformer les prédictions en score continu entre 0 et 1 (optionnel mais recommandé)
# Ici on suppose que les notes vont de 1 à 5
binary_scores = [(pred - 1)/4 if pred is not None else 0.5 for pred in test_predictions]  # 0.5 si None

# Calculer l'AUC
auc_llm = roc_auc_score(binary_test_labels, binary_scores)
print(f"AUC test LLM : {auc_llm:.4f}")

AUC test LLM : 0.9052


In [ ]:
from sklearn.model_selection import train_test_split

# Split directement sur le DataFrame déjà formaté
calibration_formatted, real_test_formatted = train_test_split(
    test_formatted, test_size=0.2, random_state=42
)

# Vérifier
print(calibration_formatted.head())
print(real_test_formatted.head())

                                                 prompt completion
5100  You are a rating prediction expert. Based on t...          4
501   You are a rating prediction expert. Based on t...          5
1643  You are a rating prediction expert. Based on t...          5
2858  You are a rating prediction expert. Based on t...          5
5976  You are a rating prediction expert. Based on t...          5
                                                 prompt completion
5992  You are a rating prediction expert. Based on t...          2
4466  You are a rating prediction expert. Based on t...          5
1575  You are a rating prediction expert. Based on t...          3
4787  You are a rating prediction expert. Based on t...          4
3538  You are a rating prediction expert. Based on t...          4


In [ ]:
# Si tes notes sont censées être des entiers 1-5
calibration_formatted['completion'] = calibration_formatted['completion'].astype(int)

# Calcul du fallback
fallback_mean = round(calibration_formatted['completion'].mean())

In [ ]:
from tqdm import tqdm
import torch
import re

# Fonction pour extraire la note (1-5) d'une prédiction
def clean_prediction(pred_text, fallback=None):
    """
    Extraire le chiffre 1-5 après le dernier '### Response:'.
    Retourne fallback si aucun chiffre valide n'est trouvé.
    """
    if "### Response:" in pred_text:
        pred_text = pred_text.split("### Response:")[-1].strip()

    digits = re.findall(r"\b[1-5]\b", pred_text)
    if digits:
        return int(digits[0])

    return fallback  # fallback si pas de chiffre valide

# Liste pour stocker les scores de non-conformité (erreurs absolues)
nonconformity_scores = []

max_retries = 3
batch_size = 30

# Utiliser la moyenne des notes du calibration set comme fallback
fallback_mean = round(calibration_formatted['completion'].mean())

# Boucle sur les batches du jeu de calibration
for start_idx in tqdm(range(0, len(calibration_formatted), batch_size), desc="Calibration inference batches"):
    batch = calibration_formatted.iloc[start_idx:start_idx + batch_size]
    prompts = []

    # Préparer les prompts
    for prompt_text in batch['prompt']:
        if "### Response:" in prompt_text:
            prompt_text = prompt_text.split("### Response:")[0].strip() + "### Response:\n"
        prompts.append(prompt_text)

    # Tokenisation et passage sur GPU
    inputs = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Génération des prédictions
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            generation_config=generation_config
        )

    # Parcours des sorties
    for i, output in enumerate(outputs):
        prediction = tokenizer.decode(output, skip_special_tokens=True)
        pred_note = clean_prediction(prediction, fallback_mean)

        # Relances si note invalide
        retries = 0
        while (pred_note not in [1, 2, 3, 4, 5]) and retries < max_retries:
            retries += 1
            with torch.no_grad():
                new_output = model.generate(
                    input_ids=inputs['input_ids'][i].unsqueeze(0),
                    attention_mask=inputs['attention_mask'][i].unsqueeze(0),
                    generation_config=generation_config
                )
            prediction = tokenizer.decode(new_output[0], skip_special_tokens=True)
            pred_note = clean_prediction(prediction, fallback_mean)

        # Vraie note pour LLM
        true_label = int(batch.iloc[i]['completion'])
        error = abs(pred_note - true_label)
        nonconformity_scores.append(error)

print("Nombre de scores de non-conformité calculés :", len(nonconformity_scores))

Calibration inference batches: 100%|██████████| 205/205 [09:37<00:00,  2.82s/it]

Nombre de scores de non-conformité calculés : 6144


In [ ]:
import numpy as np

epsilons = [0.1, 0.05, 0.01]
tau_list = [np.quantile(nonconformity_scores, 1 - epsilon) for epsilon in epsilons]

for epsilon, tau in zip(epsilons, tau_list):
    print(f"Quantile tau calibré pour epsilon={epsilon} (confiance {100*(1-epsilon)}%) : {tau:.3f}")

Quantile tau calibré pour epsilon=0.1 (confiance 90.0%) : 1.000
Quantile tau calibré pour epsilon=0.05 (confiance 95.0%) : 2.000
Quantile tau calibré pour epsilon=0.01 (confiance 99.0%) : 3.000


In [ ]:
from tqdm import tqdm
import torch

# Paramètres
batch_size = 30
max_retries = 3

# fallback_mean = moyenne des labels du jeu de calibration (formaté)
fallback_mean = round(calibration_formatted['completion'].astype(int).mean())

prediction_intervals_all = []  # Liste pour stocker les intervalles pour tous les epsilon
predicted_ratings = []         # Liste pour stocker les prédictions du modèle

# Boucle sur le test réel issu du split
for start_idx in tqdm(range(0, len(real_test_formatted), batch_size), desc="Test inference batches"):
    batch = real_test_formatted.iloc[start_idx:start_idx + batch_size]
    prompts = batch['prompt'].tolist()

    # Tokenisation et passage sur GPU
    inputs = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            generation_config=generation_config
        )

    for i, output in enumerate(outputs):
        prediction = tokenizer.decode(output, skip_special_tokens=True)
        pred_note = clean_prediction(prediction, fallback_mean)

        # Relances si note invalide
        retries = 0
        while (pred_note not in [1, 2, 3, 4, 5]) and retries < max_retries:
            retries += 1
            with torch.no_grad():
                new_output = model.generate(
                    input_ids=inputs['input_ids'][i].unsqueeze(0),
                    attention_mask=inputs['attention_mask'][i].unsqueeze(0),
                    generation_config=generation_config
                )
            prediction = tokenizer.decode(new_output[0], skip_special_tokens=True)
            pred_note = clean_prediction(prediction, fallback_mean)

        predicted_ratings.append(pred_note)  # <-- sauvegarde la prédiction

        # Calcul des intervalles pour chaque tau
        intervals_for_example = []
        for tau in tau_list:  # tau_list défini précédemment à partir du calibration set
            lower = max(1, int(round(pred_note - tau)))
            upper = min(5, int(round(pred_note + tau)))
            intervals_for_example.append((lower, upper))

        prediction_intervals_all.append(intervals_for_example)

print("Intervalles conformes et prédictions générés pour le test réel.")

Test inference batches: 100%|██████████| 52/52 [02:24<00:00,  2.78s/it]

Intervalles conformes et prédictions générés pour le test réel.


In [ ]:
import numpy as np

# epsilons choisis
epsilons = [0.1, 0.05, 0.01]

# Calcul des tau à partir des nonconformity scores
tau_list = [np.quantile(nonconformity_scores, 1 - epsilon) for epsilon in epsilons]

# Générer les intervalles conformes
prediction_intervals_all = []
for pred in test_predictions:
    intervals_for_pred = []
    for tau in tau_list:
        lower = max(1, int(round(pred - tau)))
        upper = min(5, int(round(pred + tau)))
        intervals_for_pred.append((lower, upper))
    prediction_intervals_all.append(intervals_for_pred)

In [ ]:
epsilons = [0.1, 0.05, 0.01]

for eps_idx, epsilon in enumerate(epsilons):
    count_in_interval = 0
    for i in range(len(true_labels)):
        lower, upper = prediction_intervals_all[i][eps_idx]  # intervalle pour cet epsilon
        if lower <= true_labels[i] <= upper:
            count_in_interval += 1
    coverage = count_in_interval / len(true_labels)
    print(f"Couverture empirique pour epsilon={epsilon}: {coverage*100:.2f}%")

Couverture empirique pour epsilon=0.1: 98.36%
Couverture empirique pour epsilon=0.05: 99.92%
Couverture empirique pour epsilon=0.01: 99.97%


In [ ]:
epsilons = [0.1, 0.05, 0.01]

for eps_idx, epsilon in enumerate(epsilons):
    total_width = 0
    for i in range(len(prediction_intervals_all)):
        lower, upper = prediction_intervals_all[i][eps_idx]  # intervalle pour cet epsilon
        total_width += (upper - lower)  # largeur de l'intervalle
    mean_width = total_width / len(prediction_intervals_all)
    print(f"Largeur moyenne des intervalles pour epsilon={epsilon}: {mean_width:.2f}")

Largeur moyenne des intervalles pour epsilon=0.1: 1.57
Largeur moyenne des intervalles pour epsilon=0.05: 2.74
Largeur moyenne des intervalles pour epsilon=0.01: 3.57


In [ ]:
epsilons = [0.1, 0.05, 0.01]

for eps_idx, epsilon in enumerate(epsilons):
    hallucination_count = 0
    for i in range(len(true_labels)):
        lower, upper = prediction_intervals_all[i][eps_idx]
        # Si la vraie note n’est pas dans l’intervalle → hallucination
        if not (lower <= true_labels[i] <= upper):
            hallucination_count += 1
    hallucination_rate = hallucination_count / len(true_labels)
    print(f"Taux d’hallucination pour epsilon={epsilon}: {hallucination_rate*100:.2f}%")

Taux d’hallucination pour epsilon=0.1: 1.64%
Taux d’hallucination pour epsilon=0.05: 0.08%
Taux d’hallucination pour epsilon=0.01: 0.03%


In [ ]:
# test_predictions et true_labels ont déjà été générés lors du calcul RMSE/MAE
# test_formatted contient les prompts et labels réels

import pandas as pd

# Créer un DataFrame pour centraliser tout
df_test = test_formatted.copy()
df_test['predicted'] = test_predictions
df_test['true_label'] = true_labels

print("Aperçu des données :")
print(df_test.head())

Aperçu des données :
                                              prompt completion  predicted  \
0  You are a rating prediction expert. Based on t...          5          5   
1  You are a rating prediction expert. Based on t...          4          5   
2  You are a rating prediction expert. Based on t...          3          3   
3  You are a rating prediction expert. Based on t...          5          5   
4  You are a rating prediction expert. Based on t...          5          5   

   true_label  
0           5  
1           4  
2           3  
3           5  
4           5  
